<a href="https://colab.research.google.com/github/Yulia0Ivanova/map_kgiop/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%22%D0%97%D0%B0%D1%87%D0%B5%D1%82_PythonUrban_kgiop_objects_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговый проект. Улицы. Объекты культурного наследия.

In [ ]:
# todo собрать установку всех необходимых модулей в одном месте
!pip install folium -U
!pip install geopandas mapclassify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# todo собрать импорты всех модулей в одном месте
import pandas as pd
import geopandas as gpd

In [ ]:
# todo указать через константу, чтобы везде была одна и таже подложка карт. Сейчас в работе название захардкожено 
TILES = "CartoDB positron"

# todo собрать все константы в одном месте
TERRITORY_NAME = 'Пушкинский район, Санкт-Петербург'
STREETS_FILE_URL = "https://drive.google.com/file/d/1bUT1E-QSbG1vpSNM2dOG2-LEVXSrPdo3/view?usp=sharing"
KGIOP_FILE_URL = "https://raw.githubusercontent.com/aeksei/python-urban/main/kgiop_objects.geojson"

In [ ]:
def get_google_drive_download_url(url: str) -> str:
    """
    Функция возвращает ссылку для скачивания с google drive.
     Например: "https://drive.google.com/file/d/1MWGOuqV76e0ubQOg8Ke0KTU3yGpkprHp/view?usp=sharing" ->
     "https://drive.google.com/uc?export=download&id=1MWGOuqV76e0ubQOg8Ke0KTU3yGpkprHp"
    """
    drive_id = url.split("/")[5]
    return f"https://drive.google.com/uc?export=download&id={drive_id}"

## Территория

### Загрузка территории из OSM (Extract)

In [ ]:
!pip install osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import osmnx as ox

In [ ]:
# todo указать любой район Санкт-Петербург из OSM https://wiki.openstreetmap.org/wiki/RU:%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B
TERRITORY_NAME = "Пушкинский район, Санкт-Петербург"  # например, 'Адмиралтейский район, Санкт-Петербург'

territory = ox.geocode_to_gdf(TERRITORY_NAME)  # подгружаем с OSM и получаем GeoDataFrame
territory.explore(tiles=TILES)

### Обработка данных с территорией (Transform)

In [ ]:
# todo добавить столбец Название территории и удалить все остальные (кроме столбца geometry)

## Улицы

### Загрузка файла с улицами из google disk (Extract)

In [ ]:
STREETS_FILE_URL = "https://drive.google.com/file/d/1bUT1E-QSbG1vpSNM2dOG2-LEVXSrPdo3/view?usp=sharing"
url = get_google_drive_download_url(STREETS_FILE_URL)
gdf_streets = gpd.read_file(url, mask=territory)  # todo отфильтровать улицы по маске геометрии территории полученной ранее
gdf_streets

,type,id,tags,geometry
0,way,4414959,"{'highway': 'residential', 'lanes': '2', 'lit'...","LINESTRING (3386560.825 8339904.673, 3386543.1..."
1,way,4414960,"{'highway': 'residential', 'lanes': '2', 'lit'...","LINESTRING (3386296.074 8339493.502, 3386326.1..."
2,way,4414988,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3385061.151 8336249.836, 3385080.0..."
3,way,4456239,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3383520.646 8339537.624, 3383522.2..."
4,way,4460441,"{'destination': 'Россия;Москва', 'destination:...","LINESTRING (3380805.073 8358710.316, 3380674.5..."
...,...,...,...,...
8327,way,982201974,{'highway': 'service'},"LINESTRING (3387389.855 8336056.523, 3387395.9..."
8328,way,982201975,{'highway': 'service'},"LINESTRING (3399572.860 8344030.830, 3399577.6..."
8329,way,982201976,"{'access': 'private', 'highway': 'service'}","LINESTRING (3399513.438 8343712.428, 3399614.3..."
8330,way,982309906,"{'highway': 'service', 'psv': 'yes', 'surface'...","LINESTRING (3386841.039 8339837.185, 3386808.8..."


### Обработка данных с улицами (Transform)

In [ ]:
tags = pd.json_normalize(gdf_streets["tags"])
tags

,highway,lanes,lit,maxspeed,name,oneway,surface,width,foot,parking:lane:both,...,motorway,traffic_sign,lanes:forward,turn:backward,waterway,proposed:oneway,turn,footway,admin_level,boundary
0,residential,2,yes,RU:urban,бульвар Алексея Толстого,yes,asphalt,10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,residential,2,yes,RU:urban,Ахматовская улица,NaN,asphalt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,secondary,2,yes,RU:urban,Садовая улица,NaN,asphalt,NaN,no,no_stopping,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,secondary,1,yes,RU:urban,площадь Египетские Ворота,yes,asphalt,NaN,no,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,trunk,1,yes,RU:urban,NaN,yes,asphalt,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8327,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8328,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8329,service,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8330,service,NaN,NaN,NaN,NaN,NaN,asphalt,14,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8332 entries, 0 to 8331
Columns: 108 entries, highway to boundary
dtypes: object(108)
memory usage: 6.9+ MB


In [ ]:
extract_tags = [
    "name"  # todo указать в списке строку "name"
]

gdf_streets = gdf_streets.join(tags[extract_tags])
gdf_streets



,type,id,tags,geometry,name
0,way,4414959,"{'highway': 'residential', 'lanes': '2', 'lit'...","LINESTRING (3386560.825 8339904.673, 3386543.1...",бульвар Алексея Толстого
1,way,4414960,"{'highway': 'residential', 'lanes': '2', 'lit'...","LINESTRING (3386296.074 8339493.502, 3386326.1...",Ахматовская улица
2,way,4414988,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3385061.151 8336249.836, 3385080.0...",Садовая улица
3,way,4456239,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3383520.646 8339537.624, 3383522.2...",площадь Египетские Ворота
4,way,4460441,"{'destination': 'Россия;Москва', 'destination:...","LINESTRING (3380805.073 8358710.316, 3380674.5...",NaN
...,...,...,...,...,...
8327,way,982201974,{'highway': 'service'},"LINESTRING (3387389.855 8336056.523, 3387395.9...",NaN
8328,way,982201975,{'highway': 'service'},"LINESTRING (3399572.860 8344030.830, 3399577.6...",NaN
8329,way,982201976,"{'access': 'private', 'highway': 'service'}","LINESTRING (3399513.438 8343712.428, 3399614.3...",NaN
8330,way,982309906,"{'highway': 'service', 'psv': 'yes', 'surface'...","LINESTRING (3386841.039 8339837.185, 3386808.8...",NaN


In [ ]:
gdf_streets.dropna(
    how="all",  # удалить если отсутствуют значения во всех указанных столбцах
    subset=["name"],  # столбцы, в которых надо искать пропуски
    inplace=True
)
gdf_streets

,type,id,tags,geometry,name
0,way,4414959,"{'highway': 'residential', 'lanes': '2', 'lit'...","LINESTRING (3386560.825 8339904.673, 3386543.1...",бульвар Алексея Толстого
1,way,4414960,"{'highway': 'residential', 'lanes': '2', 'lit'...","LINESTRING (3386296.074 8339493.502, 3386326.1...",Ахматовская улица
2,way,4414988,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3385061.151 8336249.836, 3385080.0...",Садовая улица
3,way,4456239,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3383520.646 8339537.624, 3383522.2...",площадь Египетские Ворота
6,way,8146528,"{'highway': 'trunk', 'int_ref': 'E 105;AH8', '...","LINESTRING (3384171.263 8355479.155, 3383984.0...",Московское шоссе
...,...,...,...,...,...
8302,way,975186069,"{'foot': 'no', 'highway': 'secondary', 'lanes'...","LINESTRING (3383446.841 8341298.194, 3383434.5...",Петербургское шоссе
8303,way,976477993,"{'addr:suburb': 'Новосёлки', 'highway': 'resid...","LINESTRING (3385384.312 8333467.983, 3385339.4...",Гражданская улица
8307,way,977815421,"{'highway': 'residential', 'lanes': '2', 'lit'...","LINESTRING (3380253.285 8345329.833, 3380238.2...",Образцовая улица
8308,way,978125029,"{'highway': 'service', 'name': 'Прудовая дорог...","LINESTRING (3386163.181 8336220.104, 3386149.6...",Прудовая дорога


In [ ]:
# todo проверить наличие дубликатов по полям "id", "geometry", "name"
gdf_streets.duplicated(subset=["id", "geometry", "name"]).value_counts()

False    1807
dtype: int64

In [ ]:
gdf_streets["name"].is_unique

False

In [ ]:
gdf_streets = gdf_streets.dissolve(by="name")  # todo сгруппировать и объединить геометрии с одинаковыми названиями
gdf_streets

,geometry,type,id,tags
name,,,,
1-й Бадаевский проезд,"MULTILINESTRING ((3384252.237 8357215.464, 338...",way,46102592,"{'highway': 'unclassified', 'lanes': '1', 'lit..."
1-й проезд,"MULTILINESTRING ((3377194.348 8340527.998, 337...",way,82226373,"{'highway': 'residential', 'name': '1-й проезд..."
1-я Краснофлотская улица,"LINESTRING (3391605.791 8329942.456, 3391541.3...",way,172359215,"{'highway': 'residential', 'name': '1-я Красно..."
1-я Новая улица,"LINESTRING (3382573.573 8331290.424, 3382409.7...",way,147948621,"{'highway': 'service', 'name': '1-я Новая улиц..."
1-я Советская улица,"MULTILINESTRING ((3387364.508 8329267.010, 338...",way,31400244,"{'highway': 'residential', 'lanes': '1', 'name..."
...,...,...,...,...
улица Центральная Дорога,"LINESTRING (3390649.423 8321680.047, 3390800.4...",way,191104803,"{'highway': 'residential', 'name': 'улица Цент..."
улица Чернышевского,"LINESTRING (3390526.449 8327864.622, 3390542.8...",way,105466951,"{'highway': 'residential', 'name': 'улица Черн..."
улица Чистякова,"LINESTRING (3387046.490 8336364.198, 3387063.3...",way,193547636,"{'highway': 'tertiary', 'lanes': '2', 'lit': '..."


In [ ]:
gdf_streets.index.rename("Название улицы", inplace=True)
gdf_streets.head()

,geometry,type,id,tags
Название улицы,,,,
1-й Бадаевский проезд,"MULTILINESTRING ((3384252.237 8357215.464, 338...",way,46102592,"{'highway': 'unclassified', 'lanes': '1', 'lit..."
1-й проезд,"MULTILINESTRING ((3377194.348 8340527.998, 337...",way,82226373,"{'highway': 'residential', 'name': '1-й проезд..."
1-я Краснофлотская улица,"LINESTRING (3391605.791 8329942.456, 3391541.3...",way,172359215,"{'highway': 'residential', 'name': '1-я Красно..."
1-я Новая улица,"LINESTRING (3382573.573 8331290.424, 3382409.7...",way,147948621,"{'highway': 'service', 'name': '1-я Новая улиц..."
1-я Советская улица,"MULTILINESTRING ((3387364.508 8329267.010, 338...",way,31400244,"{'highway': 'residential', 'lanes': '1', 'name..."


In [ ]:
# todo удалить столбцы "id", "type" и "tags"
drop_columns = [
    "id",
    "type", 
    "tags",
]
gdf_streets.drop(columns=drop_columns, inplace=True)
gdf_streets.head()

,geometry
Название улицы,
1-й Бадаевский проезд,"MULTILINESTRING ((3384252.237 8357215.464, 338..."
1-й проезд,"MULTILINESTRING ((3377194.348 8340527.998, 337..."
1-я Краснофлотская улица,"LINESTRING (3391605.791 8329942.456, 3391541.3..."
1-я Новая улица,"LINESTRING (3382573.573 8331290.424, 3382409.7..."
1-я Советская улица,"MULTILINESTRING ((3387364.508 8329267.010, 338..."


In [ ]:
#COORDINATES = gdf_streets["geometry"]
#gdf_streets["Координаты"] = COORDINATES.to_crs(4326)
#gdf_streets

In [ ]:
gdf_streets.explore(tiles="CartoDB positron")

## Объекты культурного наследия

### Загрузка объектов культурного наследия

In [ ]:
# todo указать ссылку на файл из вашего github репозитория, которая начинается с https://raw.githubusercontent.com/
KGIOP_FILE_URL = "https://raw.githubusercontent.com/aeksei/python-urban/main/kgiop_objects.geojson"

gdf_kgiop_objects = gpd.read_file(KGIOP_FILE_URL, mask=territory)
gdf_kgiop_objects

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry
0,6857,Парк Екатерининский,Мост,1825,"г. Пушкин, через Рыбный канал",объект культурного наследия федерального значения,Постановление Правительства РФ № 527 от 10.07....,Памятник,POINT (3384134.895 8336228.624)
1,6955,Дома Царскосельского Дворцового правления,Кузница,1808,"г. Пушкин, Малая ул., 14а",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3384169.850 8337729.962)
2,6374,—,Конюшни дворцовые,1786-1792,"г. Павловск, Красных Зорь ул., 2; Революции ул...",объект культурного наследия федерального значения,Постановление Правительства РФ № 527 от 10.07....,Памятник,POINT (3388966.718 8330136.117)
3,6375,—,Жилой флигель полицейского управления,1830-е,"г. Павловск, Красных Зорь ул., 3",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3388870.204 8330254.332)
4,6376,—,Дом Пытниковой,19 в. кон.,"г. Павловск, Красных Зорь ул., 5",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3388846.938 8330155.525)
...,...,...,...,...,...,...,...,...,...
916,9548,Казармы и службы Лейб-гвардии Сводно-Казачьего...,Здание службы,1912,"г. Павловск, Обороны ул., 1б, лит. Ш",объект культурного наследия регионального знач...,Распоряжение КГИОП от 18.10.2018 № 423-р,Памятник,POINT (3390414.094 8327755.463)
917,9549,Казармы и службы Лейб-гвардии Сводно-Казачьего...,Здание службы,1912,"г. Павловск, Обороны ул., 1б, лит. Ц",объект культурного наследия регионального знач...,Распоряжение КГИОП от 18.10.2018 № 423-р,Памятник,POINT (3390409.307 8327885.766)
918,9550,Казармы и службы Лейб-гвардии Сводно-Казачьего...,Здание службы,1912,"г. Павловск, Обороны ул., 1б, лит. Э",объект культурного наследия регионального знач...,Распоряжение КГИОП от 18.10.2018 № 423-р,Памятник,POINT (3390433.909 8327670.359)
919,9551,Казармы и службы Лейб-гвардии Сводно-Казачьего...,Офицерский флигель,1910,"г. Павловск, Обороны ул., 1б, лит. Ю",объект культурного наследия регионального знач...,Распоряжение КГИОП от 18.10.2018 № 423-р,Памятник,POINT (3389814.638 8328128.079)


In [ ]:
#gdf_kgiop_objects = gdf_kgiop_objects.to_crs(4326)  # вычисляем в метрической проекции, а сохраняет в географической
#gdf_kgiop_objects.head()

In [ ]:
# todo оставить только столбцы object_name, object_location, object_type
drop_columns = [
    "id",
    "ensemble_name",
    "occurrence_time",
    "historical_category",
    "normative_act",
]

gdf_kgiop_objects.drop(columns=drop_columns, inplace=True)  # удаляет указанные столбцы
gdf_kgiop_objects.head()

,object_name,object_location,object_type,geometry
0,Мост,"г. Пушкин, через Рыбный канал",Памятник,POINT (3384134.895 8336228.624)
1,Кузница,"г. Пушкин, Малая ул., 14а",Памятник,POINT (3384169.850 8337729.962)
2,Конюшни дворцовые,"г. Павловск, Красных Зорь ул., 2; Революции ул...",Памятник,POINT (3388966.718 8330136.117)
3,Жилой флигель полицейского управления,"г. Павловск, Красных Зорь ул., 3",Памятник,POINT (3388870.204 8330254.332)
4,Дом Пытниковой,"г. Павловск, Красных Зорь ул., 5",Памятник,POINT (3388846.938 8330155.525)


In [ ]:
# todo переименовать столбцы object_name, object_location, object_type в русские названия
rename_columns = {
    "object_name": "Название",
    "object_location": "Местоположение",
    "object_type": "Категория объекта",
}
gdf_kgiop_objects.rename(columns=rename_columns, inplace=True)
gdf_kgiop_objects.head()

,Название,Местоположение,Категория объекта,geometry
0,Мост,"г. Пушкин, через Рыбный канал",Памятник,POINT (3384134.895 8336228.624)
1,Кузница,"г. Пушкин, Малая ул., 14а",Памятник,POINT (3384169.850 8337729.962)
2,Конюшни дворцовые,"г. Павловск, Красных Зорь ул., 2; Революции ул...",Памятник,POINT (3388966.718 8330136.117)
3,Жилой флигель полицейского управления,"г. Павловск, Красных Зорь ул., 3",Памятник,POINT (3388870.204 8330254.332)
4,Дом Пытниковой,"г. Павловск, Красных Зорь ул., 5",Памятник,POINT (3388846.938 8330155.525)


In [ ]:
gdf_kgiop_objects.explore(tiles="CartoDB positron")

In [ ]:
#Coordinates = gdf_streets["geometry"]
#gdf_streets["Coordinates"] = Coordinates.to_crs(4326)
#gdf_streets

In [ ]:
STREET_BUFFER = 100

def get_contains_kgiop_objects(street) -> int:
    """ Функция для подсчета количества объектов культурного наследия попадающих в буффер улицы"""
    return sum(gdf_kgiop_objects["geometry"].within(street.buffer(STREET_BUFFER)))


gdf_streets["contains_kgiop_objects"] = gdf_streets["geometry"].apply(get_contains_kgiop_objects)
gdf_streets.nlargest(5, "contains_kgiop_objects")

,geometry,contains_kgiop_objects
Название улицы,,
Садовая улица,"MULTILINESTRING ((3385061.151 8336249.836, 338...",42
Оранжерейная улица,"MULTILINESTRING ((3384470.813 8336624.713, 338...",27
Парковая улица,"MULTILINESTRING ((3388077.320 8332544.145, 338...",27
Дворцовая улица,"MULTILINESTRING ((3383665.595 8337133.541, 338...",25
Павловское шоссе,"MULTILINESTRING ((3385315.628 8335886.368, 338...",24


In [ ]:
gdf_streets.explore("contains_kgiop_objects", tiles="CartoDB positron")

In [ ]:
gdf_streets["density"] = gdf_streets["contains_kgiop_objects"] / gdf_streets.length

select_columns = []
gdf_streets.nlargest(5, "density")

,geometry,contains_kgiop_objects,density
Название улицы,,,
дорога Красовского,"MULTILINESTRING ((3382684.291 8331633.504, 338...",15,0.051237
Колокольная улица,"LINESTRING (3382751.740 8331897.580, 3382704.6...",5,0.043815
Руинный мост,"LINESTRING (3382887.716 8338539.831, 3382912.7...",2,0.041878
Павловский мост,"LINESTRING (3385080.065 8336238.005, 3385153.2...",3,0.034339
Каменная улица,"LINESTRING (3382416.345 8331563.973, 3382394.5...",16,0.027026


In [ ]:
# todo переименовать столбцы contains_kgiop_objects	и density в русские названия
rename_columns = {
    "contains_kgiop_objects": "Количество ОКН",
    "density": "Плотность",
}
gdf_streets.rename(columns=rename_columns, inplace=True)

In [ ]:
m = gdf_streets.explore("contains_kgiop_objects", tiles=TILES)
m = territory.explore(m=m, style_kwds={"fill": False, "weight": 5})

m

In [ ]:
# todo сохранить датафрейм gdf_streets в файл streets_with_contains_kgiop.geojson в формате geojson
gdf_streets.to_file('streets_with_contains_kgiop.geojson', driver='GeoJSON')

In [ ]:
!pip install git+https://github.com/giswqs/leafmap.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/giswqs/leafmap.git to /tmp/pip-req-build-zz7fx3md
  Running command git clone -q https://github.com/giswqs/leafmap.git /tmp/pip-req-build-zz7fx3md
     |████████████████████████████████| 1.2 MB 9.1 MB/s 
     |████████████████████████████████| 130 kB 50.5 MB/s 
     |████████████████████████████████| 3.4 MB 57.6 MB/s 
     |████████████████████████████████| 3.8 MB 41.6 MB/s 
     |████████████████████████████████| 8.8 MB 56.4 MB/s 
     |████████████████████████████████| 46 kB 3.3 MB/s 
     |████████████████████████████████| 3.0 MB 51.7 MB/s 
     |████████████████████████████████| 99 kB 7.4 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 1.6 MB 45.5 MB/s 
     |████████████████████████████████| 9.8 MB 57.2 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 
     |████████████████████████████████

In [ ]:
!pip install keplergl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.7 MB 7.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922344 sha256=26c33289c53d138a212fe3c0d9ac7f47f3a4526a670f75de9dc95d92f657d44c
  Stored in directory: /root/.cache/pip/wheels/4b/19/e5/0852b7888ee350e9c57087ccc61eeab9daf50f5caac9c690ec
Successfully built keplergl


In [ ]:
import leafmap

In [ ]:
m = leafmap.Map()
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [ ]:
import leafmap.kepler as leafmap

In [ ]:
m = leafmap.Map(center=[59.42, 30.23], zoom=12)
in_csv = 'https://raw.githubusercontent.com/Yulia0Ivanova/map_kgiop/main/streets_with_contains_kgiop_1.csv?token=GHSAT0AAAAAABZXMHDVD4P2AATWF4EVKS2MY2NH5SA'
config = 'https://raw.githubusercontent.com/Yulia0Ivanova/map_kgiop/main/streets_with_contains_kgiop_1.geojson?token=GHSAT0AAAAAABZXMHDULAA7WHHYN2JKIJYGY2NH44A'
m.add_csv(in_csv, layer_name="data", config=config)
m

Extra data: line 1 column 4 (char 3)


Map(config={'version': 'v1', 'config': {'mapState': {'latitude': 59.42, 'longitude': 30.23, 'zoom': 12, 'beari…